In [1]:
import re

from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import ConversationChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from tqdm import notebook
import numpy as np
import pandas as pd
import os
import subprocess

ModuleNotFoundError: No module named 'langchain'

# Generation

In [18]:
template = """<s>[INST] <<SYS>>\\nYou are a helpful assistant. You will write code for the user question in C++. Always write code within markdown code blocks.
Don't write any extra text.\\n<</SYS>>\\n\\nWrite a C++ program that carries out the following operation - \n{question}[/INST]\n Here is the C++ program: """

prompt = PromptTemplate(template=template, input_variables=["question"])

In [19]:
# question = "Maximum Prefix Sum possible by merging two given arrays | C ++ Program to implement the above approach ; Stores the maximum prefix sum of the array A [ ] ; Traverse the array A [ ] ; Stores the maximum prefix sum of the array B [ ] ; Traverse the array B [ ] ; Driver code"

In [20]:
# program = "#include <bits/stdc++.h>\nusing namespace std ;\nint maxPresum ( vector < int > a , vector < int > b )"

In [29]:
MODEL_PATH = "./models/codellama-7b-instruct.Q4_K_M.gguf"
CODE_GENERATION_PATH = "./generations/"
STATIC_ANALYSIS_REPORTS_PATH = "./reports"

In [22]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
n_gpu_layers = 1
n_batch = 512

llm = LlamaCpp(
    model_path=MODEL_PATH,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    max_tokens=8192,
    temperature = 0.5,
    top_p = 1,
    f16_kv=True,
    # callback_manager=callback_manager,
    verbose=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ./models/codellama-7b-instruct.Q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32016,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_K     [  4096,  4096,

In [23]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [30]:
if not os.path.exists(CODE_GENERATION_PATH):
    os.makedirs(CODE_GENERATION_PATH)
if not os.path.exists(STATIC_ANALYSIS_REPORTS_PATH):
    os.makedirs(STATIC_ANALYSIS_REPORTS_PATH)

In [25]:
def extract_code(response):
    lines = response.split('\n')
    code = '\n'.join(lines)
    return code

def save_file(path, code):
    with open(path, 'w') as file:
        file.write(code)

def generate_code(df, save_path, n_passes=5):
    for index, row in notebook.tqdm(df.iterrows(), total=df.shape[0]):
        question = row['question']
        for p in range(n_passes):
            response = llm_chain.run(question)
            code = extract_code(response)
            save_file(f"{save_path}/question-{index}-pass-{p}.cpp", code)

In [26]:
# response = llm_chain.run({"question": question, "program": program})

In [27]:
train_df = pd.read_csv('./data/train.csv', index_col=0)
train_df.head()

,text,code,question
id,,,
0,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Maximum Prefix Sum possible by merging two giv...
1,Check if a number can be represented as sum of...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if a number can be represented as sum of...
2,Generate an N | C ++ program for the above app...,#include <bits/stdc++.h> NEW_LINE using namesp...,Generate an N.
3,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Nth natural number after removing all numbers ...
4,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if an integer is rotation of another giv...


In [ ]:
generate_code(train_df, CODE_GENERATION_PATH, n_passes=1)

# Evaluation

In [51]:
for filename in notebook.tqdm(os.listdir(CODE_GENERATION_PATH)):
    if filename.endswith(".cpp"):
        file_path = os.path.join(CODE_GENERATION_PATH, filename)

        report_file_path_text = os.path.join(STATIC_ANALYSIS_REPORTS_PATH, f"{filename.split()[0]}.txt")
        report_file_path_xml = os.path.join(STATIC_ANALYSIS_REPORTS_PATH, f"{filename.split()[0]}.xml")

        proc = subprocess.Popen(['cppcheck', '--xml', file_path], stdout=open(report_file_path_text, 'w'), stderr=open(report_file_path_xml, 'w'))
        proc.wait()

  0%|          | 0/5 [00:00<?, ?it/s]